In [1]:
datalist = [140, 162, 255, 301, 425, 672, 685, 1127, 1150, 1197, 1248, 1523, 1626, 1648, 1756, 1816, 2790, 2799, 2835, 2821, 1807, 1786, 1646, 1624, 1577, 1296, 1135 ,1074, 1039, 356, 352, 321, 287]

In [7]:
def getJenksBreaks(datalist, numclass):
    
    datalist.sort()
    # fill the matrices with data+1 arrays of numclass 0s.
    
    # mat1 = lower_class_limits
    # mat2 = variance_combinations
    mat1 = []
    mat2 = []
    for i in range(0, len(datalist) + 1):
        temp1 = []
        temp2 = []
        for j in range(0, numclass + 1):
            temp1.append(0)
            temp2.append(0)
        mat1.append(temp1)
        mat2.append(temp2)

    for i in range(1, numclass + 1):
        mat1[1][i] = 1
        mat2[1][i] = 0
        for j in range(2, len(datalist) + 1):
            mat2[j][i] = float('inf')
    
    # define Jenks matrics
    
    # v = variance
    v = 0.0
    for l in range(2, len(datalist) + 1):
        # s1 = sum, s2 = sum of squares
        s1 = 0.0
        s2 = 0.0
        w = 0.0
        for m in range(1, l + 1):
            # i3 is lower class limit
            i3 = l - m + 1
            val = float(datalist[i3 - 1])
            
            # here, we estimate variance for each potential classing of the data, for each potential number of classes
            # w is the number of data points considered so far
            
            s1 += val
            s2 += val * val
            w += 1
            v = s2 - (s1 * s1)/w
            
            # i4 is lower class limit - 1
            i4 = i3 - 1
            if i4 != 0:
                for j in range(2, numclass +1):
                    if mat2[l][j] >= (v + mat2[i4][j - 1]):
                        mat1[l][j] = i3
                        mat2[l][j] = v + mat2[i4][j - 1]
        mat1[l][1] = 1
        mat2[l][1] = v
    #print mat2
    
    # calculating Jenks break

    k = len(datalist)
    kclass = []
    for i in range(0, numclass + 1):
        kclass.append(0)
    kclass[numclass] = float(datalist[len(datalist) - 1])
    countNum = numclass

    while countNum >=2:
        id = int((mat1[k][countNum]) - 2)
        kclass[countNum - 1] = datalist[id]
        k = int((mat1[k][countNum] - 1))
        countNum -= 1
        print kclass
    
    return kclass    

In [8]:
# the goodness of variance fit (GVF) is calculated by taking the difference between the squared deviations from the array mean(SDAM)
# and the squared deviations from the class means (SDCM) and dividing by the SDAM

def getGVF(datalist, numclass):

    breaks = getJenksBreaks(datalist, numclass)
    datalist.sort()
    listMean = sum(datalist)/len(datalist) 
    #print listMean 

    # calculate the sum of squared deviations from the array mean
    SDAM = 0.0 
    for i in range(0,len(datalist)): 
        sqDev = (datalist[i] - listMean)**2 
        SDAM += sqDev 

    # calculate the sum of squared deviations between the class mean
    SDCM = 0.0 
    for i in range(0,numclass): 
        if breaks[i] == 0: 
            classStart = 0 
        else:
            classStart = datalist.index(breaks[i]) 
            classStart += 1 
        classEnd = datalist.index(breaks[i+1]) 
        classList = datalist[classStart:classEnd+1]
        if len(classList) != 0:
            classMean = sum(classList)/len(classList) 
            #print classMean 

            preSDCM = 0.0 
            for j in range(0,len(classList)): 
                sqDev2 = (classList[j] - classMean)**2 
                preSDCM += sqDev2 
            SDCM += preSDCM 
            
    return (SDAM - SDCM)/SDAM 
    
# written by Drew 
# used after running getJenksBreaks() 
def classify(value, breaks): 
    for i in range(1, len(breaks)): 
        if value < breaks[i]: 
            return i 
    return len(breaks) - 1 

In [9]:
gvf = 0.0
numclass = 2
while gvf < 0.8:
    gvf = getGVF(datalist, numclass)
    numclass +=1

print datalist
print numclass, gvf

[0, 1296, 2835.0]
[0, 0, 1816, 2835.0]
[0, 685, 1816, 2835.0]
[140, 162, 255, 287, 301, 321, 352, 356, 425, 672, 685, 1039, 1074, 1127, 1135, 1150, 1197, 1248, 1296, 1523, 1577, 1624, 1626, 1646, 1648, 1756, 1786, 1807, 1816, 2790, 2799, 2821, 2835]
4 0.9195470614
